In [28]:
import os
import sys
import requests
from bs4 import BeautifulSoup
import pprint



In [29]:
import requests
import pandas as pd
import ast 
import re  # 정규 표현식 모듈 추가
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [264]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 전역 데이터프레임 초기화
final_data = pd.DataFrame()

# 게임 데이터를 처리하는 함수
def game_name(game_key, name):
    url = f"https://www.metacritic.com/game/{name}/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "Accept": "application/json",
        "X-Requested-With": "XMLHttpRequest",
    }
    response = requests.get(url, headers=headers)
    difficulty_raw = ''
    
    # 응답 확인
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        summary = soup.select("script")
        for s in summary:
            start_idx = re.search("componentType:\"GameStats\"", s.text)
            end_idx = re.search("k.footer=c;", s.text)
            if start_idx and end_idx:
                difficulty_raw += s.text[start_idx.span()[1]:end_idx.span()[0]]

    pattern = r"difficulty:\[\{.*?\}\]"
    match = re.search(pattern, difficulty_raw, re.DOTALL)

    if match:
        difficulty_raw = match.group(0)[11:]

    temp = difficulty_raw.replace("total", "'total'").replace("label", "'label'").replace("difficulty_id", "'difficulty_id'")
    final = eval(temp)

    # 데이터프레임 생성 및 처리
    df = pd.DataFrame(final)
    df_pivot = df.pivot(columns="difficulty_id", values="total").fillna(0)
    df_pivot["game"] = game_key
    df_pivot = df_pivot.rename(columns={5: "5", 4: "4", 3: "3", 2: "2", 1: "1"})
    df_pivot = df_pivot[["game", "5", "4", "3", "2", "1"]]
    df_pivot = df_pivot.groupby("game", as_index=False).sum()
    df_pivot["total"] = df_pivot[["5", "4", "3", "2", "1"]].sum(axis=1)
    df_pivot["average"] = (
        (df_pivot["5"] * 5 + df_pivot["4"] * 4 + df_pivot["3"] * 3 + df_pivot["2"] * 2 + df_pivot["1"] * 1)
        / df_pivot["total"]
    ).round(2)

    return df_pivot

# 게임 리스트
game_list = [
    ("로블록스", "roblox"),
    ("던전앤파이터", "dungeon-fighter-online"),
    ("메이플스토리", "maplestory"),
    ("월드오브워크래프트", "world-of-warcraft"),
    ("블레이드앤소울","blade-and-soul"),
    ("마비노기","mabinogi"),
    ("리그오브레전드","league-of-legends"),
    ("아레나오브발러","arena-of-valor"),
    ("도타2","dota-2"),
    ("배틀그라운드","playerunknowns-battlegrounds"),
    ("오버워치","overwatch"),
    ("발로란트","valorant"),
    ("클래시오브클랜","clash-of-clans"),
    ("클래시로얄","clash-royale"),
    ("스타크래프트","starcraft"),
    ("캔디크러쉬사가","candy-crush-saga"),
    ("마인크래프트","minecraft"),
    ("모여봐요동물의숲:뉴호라이즌","animal-crossing-new-horizons"),
    ("포켓몬스터","pokemon-scarlet")
]
# 각 게임 데이터를 처리하여 final_data에 추가
for game_key, name in game_list:
    new_game_data = game_name(game_key, name)
    final_data = pd.concat([final_data, new_game_data], ignore_index=True)

#######################이하와 합쳐보자##########################

def gamefaqs_game_name(game_key, name):
    url = f"https://gamefaqs.gamespot.com/{name}/stats"
    headers = {
        "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0',
        "Accept": "application/json",
        "X-Requested-With": "XMLHttpRequest",
    }
    response = requests.get(url, headers=headers)
    print(response)    
    difficulty_raw = ''
        
    # 응답 확인
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        summary = soup.select("script")
        for s in summary:
        # 정규식으로 { label: ... }와 Just Right 포함 부분 매칭
            match = re.search(r'\{ label:.*?Unforgiving.*?\}', s.text)
            if match:
                difficulty_raw = match.group()
    difficulty_raw
    pattern = r'{ label:"(.*?)", data: (\d+), color:"(.*?)" }'
    matches = re.findall(pattern, difficulty_raw)

    parsed_data = []
    for label, data, color in matches:
        label_text = label.split(" - ")[0]  # "Simple", "Easy", "Just Right" 추출
        parsed_data.append({"label": label_text, "data": int(data)})
    df = pd.DataFrame(parsed_data)
    difficulty_map = {
        "Simple": 1,    "Easy": 2,    "Just Right": 3,    "Tough": 4,    "Unforgiving": 5
    }
    for label in difficulty_map.keys():
        if label not in [d["label"] for d in parsed_data]:
            parsed_data.append({"label": label, "data": 0})
            
    total_data = {"5": 0, "4": 0, "3": 0, "2": 0, "1": 0}
    for _, row in df.iterrows():
        column = str(difficulty_map[row["label"]])
        total_data[column] += row["data"]
        
    other_df = df
    other_df = other_df.rename(columns={"data": "total"})
    other_df["difficulty_id"] = other_df["label"].map(difficulty_map)
    other_df = other_df.pivot(columns="difficulty_id", values="total").fillna(0)
    other_df["game"] = game_key
    other_df = other_df.rename(columns={5: "5", 4: "4", 3: "3", 2: "2", 1: "1"})
    other_df = other_df[["game", "5", "4", "3", "2", "1"]]
    other_df = other_df.groupby("game", as_index=False).sum()
    other_df["total"] = other_df[["5", "4", "3", "2", "1"]].sum(axis=1)
    other_df["average"] = (
        (other_df["5"] * 5 + other_df["4"] * 4 + other_df["3"] * 3 + other_df["2"] * 2 + other_df["1"] * 1)
        / other_df["total"]
        ).round(2)
    return other_df

gamefaqs_game_list = [
    ("브롤스타즈", "android/254094-brawl-stars"),
    # ("쿠키런킹덤",'iphone/330536-cookie-run-kingdom')
]
print(final_data)
for game_key, name in gamefaqs_game_list:
    gamefaqs_game_data = gamefaqs_game_name(game_key, name)
    final_data = pd.concat([final_data, gamefaqs_game_data], ignore_index=True)


# 최종 데이터 출력
final_data_sorted = final_data.sort_values(by="average", ascending=True)
final_data_sorted


difficulty_id            game      5       4       3       2      1   total  \
0                        로블록스   16.0    14.0   186.0    78.0   77.0   371.0   
1                      던전앤파이터    3.0    14.0    52.0     6.0    3.0    78.0   
2                      메이플스토리   10.0    59.0   170.0    58.0   22.0   319.0   
3                   월드오브워크래프트   60.0   418.0  1604.0   312.0  103.0  2497.0   
4                     블레이드앤소울    3.0     9.0    49.0     6.0    1.0    68.0   
5                        마비노기   10.0    42.0    23.0     7.0    4.0    86.0   
6                     리그오브레전드  396.0  1476.0   862.0   128.0  175.0  3037.0   
7                     아레나오브발러    1.0     5.0    20.0     3.0    1.0    30.0   
8                         도타2  328.0   596.0   206.0    26.0   23.0  1179.0   
9                      배틀그라운드   35.0   129.0    47.0     5.0    3.0   219.0   
10                       오버워치   20.0   122.0   762.0    44.0   34.0   982.0   
11                       발로란트   13.0    46.0    25.0

difficulty_id,game,5,4,3,2,1,total,average
17,모여봐요동물의숲:뉴호라이즌,19.0,16.0,362.0,1256.0,576.0,2229.0,1.94
18,포켓몬스터,7.0,7.0,101.0,158.0,74.0,347.0,2.18
0,로블록스,16.0,14.0,186.0,78.0,77.0,371.0,2.50
16,마인크래프트,54.0,497.0,3317.0,784.0,330.0,4982.0,2.83
2,메이플스토리,10.0,59.0,170.0,58.0,22.0,319.0,2.93
19,브롤스타즈,2.0,9.0,31.0,9.0,3.0,54.0,2.96
3,월드오브워크래프트,60.0,418.0,1604.0,312.0,103.0,2497.0,3.01
10,오버워치,20.0,122.0,762.0,44.0,34.0,982.0,3.05
7,아레나오브발러,1.0,5.0,20.0,3.0,1.0,30.0,3.07
1,던전앤파이터,3.0,14.0,52.0,6.0,3.0,78.0,3.10
